# Download and Extract the Data
ModelNet10 dataset is a part of ModelNet40 dataset. The dataset consists 4,899 pre-aligned shapes from 10 categories.

The 10 categories includes: 
bathtub, bed, chair, desk, dresser, monitor, night_stand, sofa, table, toilet

In [ ]:
#!pip install path.py;

In [33]:
import numpy as np
import math
import random
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import scipy.spatial.distance
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import open3d as o3d
from path import Path
import plotly.graph_objects as go
import plotly.express as px
random.seed = 42

In [3]:
path = Path("ModelNet10")
folders = [dir for dir in sorted(os.listdir(path)) if os.path.isdir(path/dir)]
classes = {folder: i for i, folder in enumerate(folders)};
classes

{'bathtub': 0,
 'bed': 1,
 'chair': 2,
 'desk': 3,
 'dresser': 4,
 'monitor': 5,
 'night_stand': 6,
 'sofa': 7,
 'table': 8,
 'toilet': 9}

In [4]:
bed_train = os.listdir(path+'/bed/train')
bed_test = os.listdir(path+'/bed/test')
print('Number of bed files for training: ', len(bed_train))
print('Number of bed files for testing: ', len(bed_test))

Number of bed files for training:  516
Number of bed files for testing:  101


The files present in the ModelNet dataset are of .off extension

In [5]:
def read_off(file):
    if 'OFF' != file.readline().strip():
        raise('Not a valid OFF header')
    n_verts, n_faces, __ = tuple([int(s) for s in file.readline().strip().split(' ')])
    verts = [[float(s) for s in file.readline().strip().split(' ')] for i_vert in range(n_verts)]
    faces = [[int(s) for s in file.readline().strip().split(' ')][1:] for i_face in range(n_faces)]
    return verts, faces

In [6]:
with open(path/"bed/train/bed_0001.off", 'r') as f:
  verts, faces = read_off(f)

In [7]:

i,j,k = np.array(faces).T
x,y,z = np.array(verts).T

In [8]:
len(x)

2095

In [9]:
def visualize_rotate(data):
    x_eye, y_eye, z_eye = 1.25, 1.25, 0.8
    frames=[]

    def rotate_z(x, y, z, theta):
        w = x+1j*y
        return np.real(np.exp(1j*theta)*w), np.imag(np.exp(1j*theta)*w), z

    for t in np.arange(0, 10.26, 0.1):
        xe, ye, ze = rotate_z(x_eye, y_eye, z_eye, -t)
        frames.append(dict(layout=dict(scene=dict(camera=dict(eye=dict(x=xe, y=ye, z=ze))))))
    fig = go.Figure(data=data,
                    layout=go.Layout(
                        updatemenus=[dict(type='buttons',
                                    showactive=False,
                                    y=1,
                                    x=0.8,
                                    xanchor='left',
                                    yanchor='bottom',
                                    pad=dict(t=45, r=10),
                                    buttons=[dict(label='Play',
                                                    method='animate',
                                                    args=[None, dict(frame=dict(duration=50, redraw=True),
                                                                    transition=dict(duration=0),
                                                                    fromcurrent=True,
                                                                    mode='immediate'
                                                                    )]
                                                    )
                                            ]
                                    )
                                ]
                    ),
                    frames=frames
            )

    return fig

In [34]:
visualize_rotate([go.Mesh3d(x=x, y=y, z=z, color='lightpink', opacity=0.50, i=i,j=j,k=k)]).show()

In [35]:
def pcshow(xs,ys,zs):
    data=[go.Scatter3d(x=xs, y=ys, z=zs,
                                   mode='markers')]
    fig = visualize_rotate(data)
    fig.update_traces(marker=dict(size=2,
                      line=dict(width=2,
                      color='DarkSlateGrey')),
                      selector=dict(mode='markers'))
    fig.show()

In [36]:
pcshow(x,y,z)

In [37]:
len(verts)

2095

# Sample Points


In [38]:
class PointSampler(object):
    def __init__(self, output_size):
        assert isinstance(output_size, int)
        self.output_size = output_size
    
    def triangle_area(self, pt1, pt2, pt3):
        side_a = np.linalg.norm(pt1 - pt2)
        side_b = np.linalg.norm(pt2 - pt3)
        side_c = np.linalg.norm(pt3 - pt1)
        s = 0.5 * ( side_a + side_b + side_c)
        return max(s * (s - side_a) * (s - side_b) * (s - side_c), 0)**0.5

    def sample_point(self, pt1, pt2, pt3):
        # barycentric coordinates on a triangle
        # https://mathworld.wolfram.com/BarycentricCoordinates.html
        s, t = sorted([random.random(), random.random()])
        f = lambda i: s * pt1[i] + (t-s)*pt2[i] + (1-t)*pt3[i]
        return (f(0), f(1), f(2))
        
    
    def __call__(self, mesh):
        verts, faces = mesh
        verts = np.array(verts)
        areas = np.zeros((len(faces)))

        for i in range(len(areas)):
            areas[i] = (self.triangle_area(verts[faces[i][0]],
                                           verts[faces[i][1]],
                                           verts[faces[i][2]]))
            
        sampled_faces = (random.choices(faces, 
                                      weights=areas,
                                      cum_weights=None,
                                      k=self.output_size))
        
        sampled_points = np.zeros((self.output_size, 3))

        for i in range(len(sampled_faces)):
            sampled_points[i] = (self.sample_point(verts[sampled_faces[i][0]],
                                                   verts[sampled_faces[i][1]],
                                                   verts[sampled_faces[i][2]]))
        
        return sampled_points
    

In [39]:
pointcloud = PointSampler(3000)((verts, faces))

In [40]:
pcshow(*pointcloud.T)

In [41]:
class Normalize(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2
        
        norm_pointcloud = pointcloud - np.mean(pointcloud, axis=0) 
        norm_pointcloud /= np.max(np.linalg.norm(norm_pointcloud, axis=1))

        return  norm_pointcloud

In [42]:
norm_pointcloud = Normalize()(pointcloud)

In [43]:
pcshow(*norm_pointcloud.T)

# Augmentations

In [44]:
class RandRotation_z(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        theta = random.random() * 2. * math.pi
        rot_matrix = np.array([[ math.cos(theta), -math.sin(theta),    0],
                               [ math.sin(theta),  math.cos(theta),    0],
                               [0,                             0,      1]])
        
        rot_pointcloud = rot_matrix.dot(pointcloud.T).T
        return  rot_pointcloud
    
class RandomNoise(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        noise = np.random.normal(0, 0.02, (pointcloud.shape))
    
        noisy_pointcloud = pointcloud + noise
        return  noisy_pointcloud

In [45]:
rot_pointcloud = RandRotation_z()(norm_pointcloud)
noisy_rot_pointcloud = RandomNoise()(rot_pointcloud)

In [46]:
pcshow(*noisy_rot_pointcloud.T)

# ToTensor

In [47]:
class ToTensor(object):
    def __call__(self, pointcloud):
        assert len(pointcloud.shape)==2

        return torch.from_numpy(pointcloud)

In [48]:
ToTensor()(noisy_rot_pointcloud)

tensor([[ 0.4696, -0.0251,  0.0227],
        [ 0.0703,  0.0880, -0.1052],
        [ 0.3337, -0.1706,  0.1558],
        ...,
        [ 0.4970,  0.7283, -0.2124],
        [-0.3046,  0.6139, -0.1051],
        [ 0.3229, -0.1279,  0.0974]], dtype=torch.float64)

In [49]:
def default_transforms():
    return transforms.Compose([
                                PointSampler(1024),
                                Normalize(),
                                ToTensor()
                              ])

# Dataset

In [50]:
class PointCloudData(Dataset):
    def __init__(self, root_dir, valid=False, folder="train", transform=default_transforms()):
        self.root_dir = root_dir
        folders = [dir for dir in sorted(os.listdir(root_dir)) if os.path.isdir(root_dir/dir)]
        self.classes = {folder: i for i, folder in enumerate(folders)}
        self.transforms = transform if not valid else default_transforms()
        self.valid = valid
        self.files = []
        for category in self.classes.keys():
            new_dir = root_dir/Path(category)/folder
            for file in os.listdir(new_dir):
                if file.endswith('.off'):
                    sample = {}
                    sample['pcd_path'] = new_dir/file
                    sample['category'] = category
                    self.files.append(sample)

    def __len__(self):
        return len(self.files)

    def __preproc__(self, file):
        verts, faces = read_off(file)
        if self.transforms:
            pointcloud = self.transforms((verts, faces))
        return pointcloud

    def __getitem__(self, idx):
        pcd_path = self.files[idx]['pcd_path']
        category = self.files[idx]['category']
        with open(pcd_path, 'r') as f:
            pointcloud = self.__preproc__(f)
        return {'pointcloud': pointcloud, 
                'category': self.classes[category]}

In [51]:
train_transforms = transforms.Compose([
                    PointSampler(1024),
                    Normalize(),
                    RandRotation_z(),
                    RandomNoise(),
                    ToTensor()
                    ])

In [52]:
train_ds = PointCloudData(path, transform=train_transforms)
valid_ds = PointCloudData(path, valid=True, folder='test', transform=train_transforms)

In [53]:
inv_classes = {i: cat for cat, i in train_ds.classes.items()};
inv_classes

{0: 'bathtub',
 1: 'bed',
 2: 'chair',
 3: 'desk',
 4: 'dresser',
 5: 'monitor',
 6: 'night_stand',
 7: 'sofa',
 8: 'table',
 9: 'toilet'}

In [54]:
print('Train dataset size: ', len(train_ds))
print('Valid dataset size: ', len(valid_ds))
print('Number of classes: ', len(train_ds.classes))
print('Sample pointcloud shape: ', train_ds[0]['pointcloud'].size())
print('Class: ', inv_classes[train_ds[0]['category']])

Train dataset size:  3991
Valid dataset size:  908
Number of classes:  10
Sample pointcloud shape:  torch.Size([1024, 3])
Class:  bathtub


In [55]:
train_loader = DataLoader(dataset=train_ds, batch_size=32, shuffle=True)
valid_loader = DataLoader(dataset=valid_ds, batch_size=64)


In [56]:
class Tnet(nn.Module):
   def __init__(self, k=3):
      super().__init__()
      self.k=k
      self.conv1 = nn.Conv1d(k,64,1)
      self.conv2 = nn.Conv1d(64,128,1)
      self.conv3 = nn.Conv1d(128,1024,1)
      self.fc1 = nn.Linear(1024,512)
      self.fc2 = nn.Linear(512,256)
      self.fc3 = nn.Linear(256,k*k)

      self.bn1 = nn.BatchNorm1d(64)
      self.bn2 = nn.BatchNorm1d(128)
      self.bn3 = nn.BatchNorm1d(1024)
      self.bn4 = nn.BatchNorm1d(512)
      self.bn5 = nn.BatchNorm1d(256)
       

   def forward(self, input):
      # input.shape == (bs,n,3)
      bs = input.size(0)
      xb = F.relu(self.bn1(self.conv1(input)))
      xb = F.relu(self.bn2(self.conv2(xb)))
      xb = F.relu(self.bn3(self.conv3(xb)))
      pool = nn.MaxPool1d(xb.size(-1))(xb)
      flat = nn.Flatten(1)(pool)
      xb = F.relu(self.bn4(self.fc1(flat)))
      xb = F.relu(self.bn5(self.fc2(xb)))
      
      #initialize as identity
      init = torch.eye(self.k, requires_grad=True).repeat(bs,1,1)
      if xb.is_cuda:
        init=init.cuda()
      matrix = self.fc3(xb).view(-1,self.k,self.k) + init
      return matrix

In [57]:
class Transform(nn.Module):
   def __init__(self):
        super().__init__()
        self.input_transform = Tnet(k=3)
        self.feature_transform = Tnet(k=64)
        self.conv1 = nn.Conv1d(3,64,1)

        self.conv2 = nn.Conv1d(64,128,1)
        self.conv3 = nn.Conv1d(128,1024,1)
       

        self.bn1 = nn.BatchNorm1d(64)
        self.bn2 = nn.BatchNorm1d(128)
        self.bn3 = nn.BatchNorm1d(1024)
       
   def forward(self, input):
        matrix3x3 = self.input_transform(input)
        # batch matrix multiplication
        xb = torch.bmm(torch.transpose(input,1,2), matrix3x3).transpose(1,2)

        xb = F.relu(self.bn1(self.conv1(xb)))

        matrix64x64 = self.feature_transform(xb)
        xb = torch.bmm(torch.transpose(xb,1,2), matrix64x64).transpose(1,2)

        xb = F.relu(self.bn2(self.conv2(xb)))
        xb = self.bn3(self.conv3(xb))
        xb = nn.MaxPool1d(xb.size(-1))(xb)
        output = nn.Flatten(1)(xb)
        return output, matrix3x3, matrix64x64

class PointNet(nn.Module):
    def __init__(self, classes = 10):
        super().__init__()
        self.transform = Transform()
        self.fc1 = nn.Linear(1024, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, classes)
        

        self.bn1 = nn.BatchNorm1d(512)
        self.bn2 = nn.BatchNorm1d(256)
        self.dropout = nn.Dropout(p=0.3)
        self.logsoftmax = nn.LogSoftmax(dim=1)

    def forward(self, input):
        xb, matrix3x3, matrix64x64 = self.transform(input)
        xb = F.relu(self.bn1(self.fc1(xb)))
        xb = F.relu(self.bn2(self.dropout(self.fc2(xb))))
        output = self.fc3(xb)
        return self.logsoftmax(output), matrix3x3, matrix64x64


In [58]:
def pointnetloss(outputs, labels, m3x3, m64x64, alpha = 0.0001):
    criterion = torch.nn.NLLLoss()
    bs=outputs.size(0)
    id3x3 = torch.eye(3, requires_grad=True).repeat(bs,1,1)
    id64x64 = torch.eye(64, requires_grad=True).repeat(bs,1,1)
    if outputs.is_cuda:
        id3x3=id3x3.cuda()
        id64x64=id64x64.cuda()
    diff3x3 = id3x3-torch.bmm(m3x3,m3x3.transpose(1,2))
    diff64x64 = id64x64-torch.bmm(m64x64,m64x64.transpose(1,2))
    return criterion(outputs, labels) + alpha * (torch.norm(diff3x3)+torch.norm(diff64x64)) / float(bs)

In [59]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [60]:
pointnet = PointNet()
#pointnet.to(device);

In [61]:
pointnet.load_state_dict(torch.load('save.pth'))
pointnet.eval();
pointnet

FileNotFoundError: [Errno 2] No such file or directory: 'save.pth'

In [ ]:
all_preds = []
all_labels = []
with torch.no_grad():
    for i, data in enumerate(valid_loader):
        print('Batch [%4d / %4d]' % (i+1, len(valid_loader)))
                   
        inputs, labels = data['pointcloud'].float(), data['category']
        print(inputs.shape)
        outputs, __, __ = pointnet(inputs.transpose(1,2))
        _, preds = torch.max(outputs.data, 1)
        all_preds += list(preds.numpy())
        all_labels += list(labels.numpy())
        for j in range(15,25):
          print(all_preds[j], all_labels[j])
        break

Batch [   1 /   15]
torch.Size([64, 1024, 3])
0 0
0 0
0 0
0 0
0 0
0 0
0 0
0 0
1 0
1 0


In [ ]:
i,j,k = np.array(faces).T
x,y,z = np.array(verts).T

In [ ]:
visualize_rotate([go.Mesh3d(x=x, y=y, z=z, color='lightpink', opacity=0.50, i=i,j=j,k=k)]).show()

In [ ]:
pointnet.to(device);

In [ ]:
def predict(dir, visualization = False):
  with open(dir, 'r') as f:
    verts, faces = read_off(f)
  i,j,k = np.array(faces).T
  x,y,z = np.array(verts).T
  if(visualization):
    visualize_rotate([go.Mesh3d(x=x, y=y, z=z, color='lightpink', opacity=0.50, i=i,j=j,k=k)]).show()
  pointcloud = default_transforms()((verts, faces))
  pointcloud = pointcloud.reshape(-1,1024,3).float().to(device)
  outputs, __, __ = pointnet(pointcloud.transpose(1,2))
  _, preds = torch.max(outputs.data, 1)
  #print('Prediction: ',inv_classes[preds.item()])
  return pointcloud, outputs, preds


In [ ]:
dir = "ModelNet10/chair/train/chair_0211.off"
pointcloud, outputs, pred = predict(dir)
#pointcloud.max()

Prediction:  chair


In [ ]:
criterion = torch.nn.CrossEntropyLoss()
def fgsm_attack(model, criterion, point, label, eps) :
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    labels = torch.Tensor([0,0,0,0,0,0,0,0,0,0]).reshape(-1,10)
    labels[0][label] = 1
    model = model.to(device)
    point = point.to(device)
    labels = labels.to(device)
    point.requires_grad = True

    outputs, _, _= model(point.transpose(1,2))
    model.zero_grad()
    cost = criterion(outputs, labels)
    cost.backward()
    
    attack_data = point + eps*point.grad.sign()

    #print(point.grad.sign())
    return attack_data

In [ ]:
def attack(model, criterion, point, label, eps, pointcloud_form=False):
  new_points = fgsm_attack(model, criterion, point, label, eps)
  outputs, __, __ = pointnet(new_points.transpose(1,2))
  _, preds = torch.max(outputs.data, 1)
  print('Predicted Label (PointNet): ',inv_classes[label], '| After FGSM: ',inv_classes[preds.item()])
  if (pointcloud_form):
    pointcloud_vis = new_points.detach().cpu().numpy()
    pointcloud_vis = pointcloud_vis.reshape(-1,3)
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(pointcloud_vis)
    o3d.io.write_point_cloud("after_attack_new.ply",pcd)

  #return new_points
  return preds


In [ ]:
attack(pointnet, criterion, pointcloud, 2, 0.2)

Predicted Label (PointNet):  chair | After FGSM:  night_stand


In [ ]:
dir = "chair/train/chair_0211.off"
with open("ModelNet10/"+dir, 'r') as f:
  verts, faces = read_off(f)
i,j,k = np.array(faces).T
x,y,z = np.array(verts).T
pointcloud_bef = default_transforms()((verts, faces))
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pointcloud_bef)
o3d.io.write_point_cloud("bef_attack.ply",pcd)

True

In [ ]:
c = "chair"
dir = "ModelNet10/"+c+"/test/"
files = os.listdir(dir)
count = 0
for f in files:
  pointcloud, outputs, pred = predict(dir+f)
  pred_attack = attack(pointnet, criterion, pointcloud, 2, 0.2)
  if(pred==pred_attack): count += 1

Predicted Label (PointNet):  chair | After FGSM:  night_stand
Predicted Label (PointNet):  chair | After FGSM:  toilet
Predicted Label (PointNet):  chair | After FGSM:  night_stand
Predicted Label (PointNet):  chair | After FGSM:  night_stand
Predicted Label (PointNet):  chair | After FGSM:  dresser
Predicted Label (PointNet):  chair | After FGSM:  dresser
Predicted Label (PointNet):  chair | After FGSM:  night_stand
Predicted Label (PointNet):  chair | After FGSM:  toilet
Predicted Label (PointNet):  chair | After FGSM:  night_stand
Predicted Label (PointNet):  chair | After FGSM:  chair
Predicted Label (PointNet):  chair | After FGSM:  night_stand
Predicted Label (PointNet):  chair | After FGSM:  night_stand
Predicted Label (PointNet):  chair | After FGSM:  night_stand
Predicted Label (PointNet):  chair | After FGSM:  dresser
Predicted Label (PointNet):  chair | After FGSM:  dresser
Predicted Label (PointNet):  chair | After FGSM:  night_stand
Predicted Label (PointNet):  chair | Aft

UnicodeDecodeError: ignored

In [ ]:
fgsm_dataset = PointCloudData(path, valid=True,  folder='test', transform=train_transforms)
fgsm_dataloader = DataLoader(fgsm_dataset, 32, shuffle=False)
pointnet.train()


PointNet(
  (transform): Transform(
    (input_transform): Tnet(
      (conv1): Conv1d(3, 64, kernel_size=(1,), stride=(1,))
      (conv2): Conv1d(64, 128, kernel_size=(1,), stride=(1,))
      (conv3): Conv1d(128, 1024, kernel_size=(1,), stride=(1,))
      (fc1): Linear(in_features=1024, out_features=512, bias=True)
      (fc2): Linear(in_features=512, out_features=256, bias=True)
      (fc3): Linear(in_features=256, out_features=9, bias=True)
      (bn1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn3): BatchNorm1d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn4): BatchNorm1d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (bn5): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (feature_transform): Tnet(
      (conv1): Conv1d(64, 64, kernel_size=(1,

In [ ]:
total_accu = 0
total_data_no = 0
class_wise_pred = torch.zeros((10))
class_wise_target = torch.zeros((10))

for i, data in enumerate(fgsm_dataloader):
  labels = data['category']
  #labels = labels.reshape(-1,1)
  input_cloud = data['pointcloud']
  input_cloud = input_cloud.type(torch.FloatTensor)
  
  preds = attack(pointnet, criterion, input_cloud, labels, eps=0.01, j=i)
  #print(preds, labels)
  acc = torch.sum(preds.detach().cpu() == labels) / preds.shape[0]
  total_accu += acc.item() * preds.shape[0]
  total_data_no += preds.shape[0]
  print(i, "Accuracy: ", acc)

  for pred, tar in zip(preds, labels):
    class_wise_pred[pred] += 1
    class_wise_target[tar] += 1

print("Accuracy: ", total_accu / total_data_no)
  

0 Accuracy:  tensor(0.1250)
1 Accuracy:  tensor(0.1875)
2 Accuracy:  tensor(0.2500)
3 Accuracy:  tensor(0.1250)
4 Accuracy:  tensor(0.4062)
5 Accuracy:  tensor(0.3750)
6 Accuracy:  tensor(0.2500)
7 Accuracy:  tensor(0.3750)
8 Accuracy:  tensor(0.)
9 Accuracy:  tensor(0.0312)
10 Accuracy:  tensor(0.1250)
11 Accuracy:  tensor(0.0938)
12 Accuracy:  tensor(0.0625)
13 Accuracy:  tensor(0.3438)
14 Accuracy:  tensor(0.2812)
15 Accuracy:  tensor(0.3125)
16 Accuracy:  tensor(0.4062)
17 Accuracy:  tensor(0.)
18 Accuracy:  tensor(0.0312)
19 Accuracy:  tensor(0.1875)
20 Accuracy:  tensor(0.1875)
21 Accuracy:  tensor(0.1562)
22 Accuracy:  tensor(0.1562)
23 Accuracy:  tensor(0.0938)
24 Accuracy:  tensor(0.0938)
25 Accuracy:  tensor(0.1875)
26 Accuracy:  tensor(0.1250)
27 Accuracy:  tensor(0.1250)
28 Accuracy:  tensor(0.0833)
Accuracy:  0.18061674012092765


In [ ]:
class_wise_accu = class_wise_pred / class_wise_target